In [1]:
import pandas as pd
import logging
from typing import List
import numpy as np
import gc

In [11]:
df = pd.read_stata('/home/stanoo/Projects/data/hc/hc-2021-22.dta')

In [12]:
to_keep = ['court_name', 'date_dd', 'date_mon', 'date_yyyy', 'caseid_type', 'caseid_no',
       'filed_dd', 'filed_mon', 'filed_yyyy', 'origcourt', 'appeal_type',
       'appeal_no', 'appeal_yyyy', 'case_type', 'bench_1', 'bench_2',
       'bench_3', 'bench_4', 'bench_5', 'bench_6', 'bench_7', 'comingfor',
       'outcome', 'reason_adj', 'next_dd', 'next_mon', 'next_yyyy',
       'fisrtparty_m', 'firstparty_f', 'firstparty_org', 'secondparty_m',
       'secondparty_f', 'secondparty_o', 'legalrep', 'p_witness', 'd_witness',
       'custody', 'other_details', 'resolved_cases']
new_df = df[to_keep]

In [13]:
del df 
gc.collect()

24240

In [16]:
years_to_keep = ['2021', '2022']
new_df = new_df[new_df['date_yyyy'].isin(years_to_keep)]

In [20]:
mon_to_keep = ['Jul','Aug', 'Sep', 'Oct','Nov', 'Dec']
df_2021 = new_df[new_df.date_mon.isin(mon_to_keep)]

In [ ]:
# mons = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun','Jul','Aug', 'Sep', 'Oct','Nov', 'Dec']

# df_2021 = new_df[new_df.date_mon.isin(mon_to_keep)]

In [26]:
df_2022 = new_df[(new_df.date_yyyy=='2022' & ~new_df.date_mon.isin(mons))]
df_2022 = new_df[(new_df.date_yyyy == '2022') & (~new_df.date_mon.isin(mons))]

In [29]:
#combine 2021 and 2022
df = pd.concat([df_2021, df_2022])

In [31]:
df.to_csv('/home/stanoo/Projects/data/hc/hc21-22.csv')

In [ ]:
df_1 = pd.read_csv('/home/stanoo/Projects/data/hc/hc-2021-23-combined.csv')


In [24]:
mons = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun','Jul','Aug', 'Sep', 'Oct','Nov', 'Dec']
df_1 = df_1[df_1.date_mon.isin(mons)]

In [7]:
df.to_csv('/home/stanoo/Projects/data/hc/2019-22.csv')

### Indicators 
#### 1 Appeals Judgements
#### 2 Certificates of urgency
#### 3 Injunctions
#### 4 Remand days
#### 5 First hearing
#### 6 Taxation - Advocate-client Bc 


In [ ]:
file_path = '/home/stanoo/Projects/data/tribunal'
df = pd.read_csv(f'{file_path}/tribunals_processed.csv')

In [ ]:
cert_df = df['outcome'].apply(lambda x: 1 if x == 'Certified Urgent' else 0)
cert_df.loc[:, 'certified'] = cert_df.groupby('case_number')['urgent'].transform('max')
urgent_cases = cert_df[cert_df['certified'] == 1]


In [18]:
# hc_stations = ['Marsabit', 'Embu', 'Chuka', 'Muranga']
# sub_df = df[df['court'].isin(hc_stations)]

## not certified application/injuctions

In [31]:
condition = (
    sub_df['comingfor'].isin(['Notice of Motion', 'Hearing of Applications', 'Hearing- Application', 'Ruling on Applications']) |
    sub_df['outcome'].isin(['Not Certified Urgent', 'Not  Certified Urgent', 'Application Heard'])
)
sub_df = sub_df.copy()

sub_df['other_applications'] = np.where(condition, 1, 0)
sub_df['application'] = sub_df.groupby('case_number')['other_applications'].transform('max')
applic_sub_df = sub_df[sub_df['application'] == 1]
applic_sub_df.to_csv(f'{file_path}/hc_application_data.csv', index=False)

applic_sub_df.groupby(['court', 'case_number'])['other_applications'].sum().sort_values(ascending=False).reset_index(name='count').sort_values(by=['court', 'count'], ascending=[True, False]).reset_index(drop=True).to_csv(f'{file_path}/hc_application_cases.csv', index=False)

## Certified Urgent applications

In [ ]:
cert_df = sub_df[sub_df['application'] == 0]
cert_df.loc[:, 'urgent'] = cert_df['outcome'].apply(lambda x: 1 if x == 'Certified Urgent' else 0)
cert_df.loc[:, 'certified'] = cert_df.groupby('case_number')['urgent'].transform('max')
urgent_cases = cert_df[cert_df['certified'] == 1]
urgent_cases.groupby(['court','case_number'])['certified'].count().reset_index(name='count').to_csv(f'{file_path}/hc_urgent_cases.csv', index=False)
urgent_cases.to_csv(f'{file_path}/hc_certificate_data.csv', index=False)

## Remand Days

In [20]:
custody_df = sub_df[(sub_df['custody'] == 1) & (sub_df['broad_case_type'] == 'Murder')]

In [ ]:
custody_df = sub_df[(sub_df['custody'] == 1) & (sub_df['broad_case_type'] == 'Murder')]
custody_df.loc[:, 'reg'] = custody_df.groupby('case_number')['registered'].transform('max')
custody_df = custody_df.copy()
custody_df = custody_df[custody_df['reg'] == 1]

In [24]:
custody_df.loc[:, 'reg'] = custody_df.groupby('case_number')['registered'].transform('max')

In [27]:
custody_df = custody_df[custody_df['resolved'] != 1]

In [29]:
custody_df.groupby(['court','case_number'])['reg'].count().sort_values(ascending=False).reset_index(name='count').to_csv(f'{file_path}/hc_custody_cases.csv', index=False)
custody_df.to_csv(f'{file_path}/hc_custody_data.csv', index=False)

In [ ]:

custody_df['resolved'] = custody_df.groupby('case_number')['concluded'].transform('max')
custody_df = custody_df[custody_df['resolved'] != 1]
custody_df.groupby(['court', 'case_type'])['reg'].count().reset_index(name='count')
custody_df.groupby(['court','case_number'])['reg'].count().sort_values(ascending=False).reset_index(name='count').to_csv(f'{file_path}/hc_custody_cases.csv', index=False)
custody_df.to_csv(f'{file_path}/hc_custody_data.csv', index=False)

In [13]:
custody_df.groupby(['court','case_number'])['reg'].count().sort_values(ascending=False)

Series([], Name: reg, dtype: int64)

## Appeals

In [50]:
# not certified application/injuctions
jd_condition = (
    sub_df['comingfor'].isin(['Judgment', 'Ruling']) &
    sub_df['outcome'].isin(["Grant Revoked", "Judgment Delivered", 
                                   "Judgment Delivered- Acquittal", "Judgment Delivered- Case Closed", 
                                   "Judgment Delivered- Convicted"])
)
sub_df = sub_df.copy()

sub_df['delivered'] = np.where(jd_condition, 1, 0)

appeals = sub_df[sub_df['broad_case_type'].isin(['Civil Appeal', 'Criminal Appeal'])]
appeals = appeals.copy()
appeals = appeals[appeals['delivered'] == 1]
appeals['judgement'] = appeals.groupby('case_number')['delivered'].transform('max')
appeals.to_csv(f'{file_path}/hc_appeal_data.csv', index=False)
(appeals.groupby(['court', 'case_number'])['judgement'].sum()
.sort_values(ascending=False).reset_index(name='count')
.sort_values(by=['court', 'count'], ascending=[True, False]).reset_index(drop=True)
.to_csv(f'{file_path}/hc_appeal_cases.csv', index=False))

## Taxations

In [53]:
taxation_condition  = sub_df['comingfor'].isin(['Taxation and Issuance of Certificates',
'Signing of Taxation''Notice of Taxation',
'Filling Bill of Cost',
'Mention of Bill of Cost',
'Taxation''Assessment of costs/Taxation',
'Assessment of costs']) | sub_df['outcome'].isin(['Taxation Date Set'])

sub_df = sub_df.copy()

sub_df['taxation'] = np.where(taxation_condition, 1, 0)
sub_df.loc[:, 'boc'] = sub_df.groupby('case_number')['taxation'].transform('max')
sub_df['resolved'] = sub_df.groupby('case_number')['concluded'].transform('max')
taxation_df = sub_df[(sub_df['boc'] == 1) & (sub_df['broad_case_type'] =='Civil Application')]
resolved_boc = taxation_df[taxation_df['resolved'] == 1]
resolved_boc.to_csv(f'{file_path}/hc_taxation_data.csv', index=False)
resolved_boc.groupby(['court', 'case_number'])['taxation'].sum().reset_index(name='count').to_csv(f'{file_path}/hc_taxation_cases.csv', index=False)

In [ ]:
resolved_boc.groupby(['court', 'case_type'])['taxation'].sum().reset_index(name='count')

In [ ]:
'Record Of Appeal Filed'

## First scheduled

In [ ]:
hearing_set =  ['Hearing Date Set (In Court)', 
'Hearing Date Set (At The Registry)', 
'Hearing Date Set',
'Hearing Date Given']
hearing_date = ['Hearing',
'Hearing - Appeals',
'Hearing- Main Suit',
'Fresh Hearing']

In [ ]:
hearing_condition = sub_df['comingfor'].isin(hearing_set) | sub_df['outcome'].isin(hearing_date)
sub_df = sub_df.copy()
sub_df['hearing'] = np.where(hearing_condition, 1, 0)

In [83]:
pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

In [ ]:
sub_df[sub_df['outcome'] == 'Directions Given'].groupby('comingfor')['comingfor'].count().sort_values(ascending=False).reset_index(name='count')

In [ ]:
sub_df.groupby('outcome')['outcome'].count().sort_values(ascending=False).reset_index(name='count')

In [ ]:
sub_df.loc[:, 'hearing'] = sub_df.groupby('case_number')['hearing'].transform('max')
sub_df['resolved'] = sub_df.groupby('case_number')['concluded'].transform('max')

In [ ]:
hearing_df = sub_df[(sub_df['hearing'] == 1) & (sub_df['broad_case_type'] =='Civil Application')]
resolved_hearing = hearing_df[hearing_df['resolved'] == 1]
resolved_hearing.to_csv(f'{file_path}/hc_hearing_data.csv', index=False)
resolved_hearing.groupby(['court', 'case_number'])['hearing'].sum().reset_index(name='count').to_csv(f'{file_path}/hc_hearing_cases.csv', index=False)

In [10]:
resolved = df[df['concluded']==1]

In [12]:
# drop dupplicates on resolved by case_number 
resolved = resolved.drop_duplicates(subset=['case_number'], keep=False)

In [13]:
tlab= resolved[resolved['court'] =='Transport Licensing Appeals Board']

In [ ]:
# rate of resolution
under_180 = tlab['age'] < 180

In [ ]:
tlab.to_csv(f'{file_path}/')